Intall Tweepy, import libraries and mount the drive. Then you have to log in with your Twitter API credentials.

In [ ]:
%%capture
!pip install tweepy


In [ ]:
import os
import tweepy as tw
import pandas as pd
import json
from datetime import datetime, timedelta
import time
import networkx as nx
import re
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
auth = tw.OAuthHandler("", "")
auth.set_access_token("", "")
api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

We collect the data from Twitter using the Tweepy library. Three queries related to teh Green Pass discussion are made. Since Twitter's API are limited, we can only collect posts that are max 7 days old. 
Notice that this process may take a while. To avoid disconnected runtimes with Colab, it is suggested to run this locally (even running this notebook with a [local runtime](https://research.google.com/colaboratory/local-runtimes.html)

In [ ]:
#this function scrap tweets using the Twitter api and creates a pandas dataframe to better manage the data
tweet_dict = {}
lista_key = ["#nogreepass", "#GreenpassObbligatorio"]

for keyword in lista_key:
  try:
    tweets = tw.Cursor(api.search, q=keyword, result_type="mixed", lang='it', exclude_mentions=False).items()
    for tweet in tweets:
      tweet_dict[tweet.id_str] = {}
      tweet_dict[tweet.id_str]["tweet_id"] = tweet.id_str
      tweet_dict[tweet.id_str]["text"] = tweet.text
      tweet_dict[tweet.id_str]["date"] = tweet.created_at
      tweet_dict[tweet.id_str]["user"] = tweet.user.id_str
      tweet_dict[tweet.id_str]["user_name"] = tweet.user.name
      tweet_dict[tweet.id_str]["user_verified"] = tweet.user.verified
      tweet_dict[tweet.id_str]["user_description"] = tweet.user.description
      tweet_dict[tweet.id_str]["in_reply_to_user"] = tweet.in_reply_to_user_id_str
      tweet_dict[tweet.id_str]["in_reply_to_tweet"] = tweet.in_reply_to_status_id_str
      tweet_dict[tweet.id_str]["in_reply_to_user_name"] = tweet.in_reply_to_screen_name
      if len (tweet.entities["user_mentions"]) > 0:
        tweet_dict[tweet.id_str]["user_mentions"] = tweet.entities["user_mentions"][0]["id"]
      if len( tweet.entities["urls"]) > 0:
        tweet_dict[tweet.id_str]["links"] = tweet.entities["urls"][0]["expanded_url"]
      else: 
        tweet_dict[tweet.id_str]["links"] = tweet.entities["urls"]
    print(len(tweet_dict.keys()))
  except tw.TweepError: 
    print("error")
    time.sleep(60)

date = datetime.today().date()
df = pd.DataFrame(data=tweet_dict)
df = df.transpose()
filename = "Twitter_GP_"+ str(date)
df.to_csv("/content/drive/MyDrive/Colab Notebooks/Twitter SNA/"+filename+".csv")
df

Rate limit reached. Sleeping for: 130
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 846
Rate limit reached. Sleeping for: 846


10446


Rate limit reached. Sleeping for: 847


12845


Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 847
Rate limit reached. Sleeping for: 850
Rate limit reached. Sleeping for: 848
Rate limit reached. Sleeping for: 850
Rate limit reached. Sleeping for: 848
Rate limit reached. Sleeping for: 848
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 847
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 848
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 846
Rate limit reached. Sleeping for: 850
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 849
Rate limit reached. Sleeping for: 850
Rate limit reached. Sleeping for: 850
Rate limit reached. Sleeping for: 847
Rate limit reached. Sleeping for: 850
Rate limit reached. Sleeping for: 851
Rate limit reached. Sleeping for: 848
Rate limit r

By runnin the previous cell on different days  we can create multiple dataframes that need to be merged togther. 
We use the Networkx library to create a set of edges that represent the connection between users replies. Since the matplot library is not able to handle this number of connections, we export the edges list into a format more suitable for Gephi. 

In [ ]:
#df_10 = pd.read_json("/content/drive/MyDrive/Colab Notebooks/Twitter SNA/Twitter_GP_2021-10-10.json")
df_17 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Twitter SNA/Twitter_GP_2021-10-17-safe_with_mentions.csv", lineterminator='\n')
df_17 = df_17.drop(["Unnamed: 0","Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1"], axis=1)
df_24 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Twitter SNA/Twitter_GP_2021-10-24-safe.csv", lineterminator='\n')
df = pd.concat([df_17, df_24])
#df = df[(df["user_mentions"].isnull()) & df["in_reply_to_tweet"].isnull()]
df



,text,date,user,user_name,user_verified,user_description,in_reply_to_user,in_reply_to_tweet,in_reply_to_user_name,links,tweet_id,user_mentions
0,Criminali. Senza alcun tipo di giustificazione...,2021-10-09 18:55:22,963073442,Chiara Appendino,True,Sindaca di Torino \ Mayor of Turin,NaN,NaN,NaN,https://twitter.com/i/web/status/1446912188657...,1446912188657057794,NaN
1,Un branco di fascisti assalta un sindacato. Qu...,2021-10-09 18:48:01,232534560,Fabio Alisei,True,Sono quello nella foto ma senza photoshop.,NaN,NaN,NaN,[],1446910335902957578,NaN
2,#nogreepass. Sono state arrestate 12 persone c...,2021-10-10 08:58:20,5893702,Sky tg24,True,"News, video, fotogallery e la diretta web 24 o...",NaN,NaN,NaN,https://twitter.com/i/web/status/1447124327380...,1447124327380439042,NaN
3,@Cartabellotta In confronto a lei #StefanoPuzz...,2021-10-17 13:10:58,287566348,Ebby,False,Giuro Solennemente di non avere buone intenzioni.,231902290.0,1.449658e+18,Cartabellotta,https://twitter.com/i/web/status/1449724621087...,1449724621087985668,2.319023e+08
4,RT @matteo_rivi: Non so più come dirlo: Puzzer...,2021-10-17 13:09:16,1420711554270511104,Deodato RIBEIRA,False,"SRV 4ever ( ok, per i più giovani..SRV means S...",NaN,NaN,NaN,[],1449724193210253312,1.254793e+18
...,...,...,...,...,...,...,...,...,...,...,...,...
14433,"Domani, 13esimo sabato consecutivo di proteste...",2021-10-15 18:00:00,3775808595,Mi-Tomorrow,False,© @pradivio • L’unico giornale gratuito del po...,NaN,NaN,NaN,https://twitter.com/i/web/status/1449072580967...,1449072580967604224,NaN
14434,Che figura di merda\n#GreenpassObbligatorio #g...,2021-10-15 17:59:32,1128224182750384128,Claudio Binnella,False,Se svolti a sinistra vai a sbattere,NaN,NaN,NaN,https://twitter.com/PeterSweden7/status/144901...,1449072465112674304,NaN
14435,Sono aumentati i certificati di malattia. \nMa...,2021-10-15 17:59:24,79211525,Kattoliko Pensiero,False,"I am an Italian Catholic, husband, teacher, bl...",NaN,NaN,NaN,[],1449072430031544322,NaN
14436,RT @CoGiafra: Perché non viene organizzata una...,2021-10-15 17:56:58,2754565029,Rosad24,False,Sturm und Drang,NaN,NaN,NaN,[],1449071816132153354,2.730966e+09


In [ ]:
import networkx as nx

df_self = df[df["in_reply_to_user"].isnull()]
df_dir= df[df["in_reply_to_user"].notnull()]
df_men= df[df["user_mentions"].notnull()]

G = nx.MultiGraph()
for index, row in df_self.iterrows():
    G.add_edge(int(row["user"]), int(row["user"]))

for index, row in df_dir.iterrows():
  G.add_edge(int(row["user"]), int(row["in_reply_to_user"]))

for index, row in df_men.iterrows():
  G.add_edge(int(row["user"]), int(row["user_mentions"]))


nx.write_edgelist(G, "/content/drive/MyDrive/Colab Notebooks/Twitter SNA/gephi_data/edges_final_17_24.csv", delimiter=',', data=False)
